In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [14]:
calhouse = fetch_california_housing()

In [15]:
sc = StandardScaler()

In [16]:
xtrain = sc.fit_transform(calhouse.data)

In [17]:
ytrain = calhouse.target

In [18]:
perm = np.random.permutation(xtrain.shape[0])

In [19]:
n, m = xtrain.shape

In [20]:
xt = xtrain[perm]
yt = ytrain[perm]

In [21]:
xt = np.c_[np.ones((n, 1)), xt]
m = m + 1

In [22]:
xt.shape

(20640, 9)

In [23]:
def fetch_batch(batch_size, offset):
    return xt[offset : (offset + batch_size)], yt[offset : (offset + batch_size)].reshape(-1, 1)

In [24]:
batch_size = 1000
n_epochs = 20
learning_rate = 0.01
n_batches = n // batch_size

In [27]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [29]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, m), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform((m, 1), -1, 1), name='theta')
prediction = tf.matmul(X, theta, name='predictions')

with tf.name_scope("loss") as scope:
    error = tf.subtract(prediction, y, name='errors')
    mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

summary = tf.summary.scalar('MinSqErr', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

print('Total steps: ', n_epochs * n_batches)

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch in range(n_batches):
            xb, yb = fetch_batch(batch_size, batch * batch_size)
            if batch % 10 == 0:
                summary_str = summary.eval(feed_dict={X: xb, y: yb})
                step = epoch * n_batches + batch
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: xb, y: yb})
    best_theta = theta.eval()

file_writer.close()

best_theta

Total steps:  400


array([[ 2.06901145],
       [ 0.69402063],
       [ 0.19208284],
       [ 0.17366128],
       [-0.13851979],
       [ 0.02303315],
       [-0.04004495],
       [-0.50773132],
       [-0.45259261]], dtype=float32)

=============== Varibales ===============================

In [ ]:
n_features = 3
X = tf.placeholder(dtype=tf.float32, shape=(None, n_features), name=X)
w1 = tf.Variable(tf.random_uniform(shape=(n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_uniform(shape=(n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b1 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')

relu1 = tf.maximum(z1, 0.0, name="relu1")
relu2 = tf.maximum(z2, 0.0, name="relu2")

output = tf.add(relu1, relu2, name="output")

In [31]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_uniform(shape=w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b1, name='z')
    return tf.maximum(z, 0.0, name="relu")